# Capítulo 8: Operações em Strings (German Strings)

## Introdução

O DuckDB usa uma representação especial para strings chamada **"German Strings"** (ou String Views), que armazena um **prefixo inline** permitindo comparações rápidas sem seguir ponteiros para a heap.

### Objetivos:
1. Entender a estrutura German String de 16 bytes
2. Comparar com strings tradicionais
3. Otimizar operações de comparação e filtragem
4. Entender Dictionary Encoding para strings
5. Benchmark de operações com strings no DuckDB

In [ ]:
!pip install duckdb pandas numpy matplotlib -q

In [ ]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import struct
import random
import string

plt.rcParams['figure.figsize'] = (12, 6)

## 8.1 Estrutura German String

```
┌─────────────────────────────────────────────────────────────────────┐
│                 GERMAN STRING LAYOUT (16 bytes)                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  ┌─────────────┬─────────────┬──────────────────────────────────┐  │
│  │ Length (4B) │ Prefix (4B) │     Pointer/Inline (8B)          │  │
│  └─────────────┴─────────────┴──────────────────────────────────┘  │
│                                                                     │
│  STRINGS CURTAS (≤ 12 bytes):                                       │
│  ┌─────────────┬────────────────────────────────────────────────┐  │
│  │ Length (4B) │           String Completa (12B)                │  │
│  └─────────────┴────────────────────────────────────────────────┘  │
│  Exemplo: "Hello" → [5, 'H', 'e', 'l', 'l', 'o', 0, 0, 0, 0, 0, 0] │
│                                                                     │
│  STRINGS LONGAS (> 12 bytes):                                       │
│  ┌─────────────┬─────────────┬──────────────────────────────────┐  │
│  │ Length (4B) │ Prefix (4B) │       Pointer to Heap (8B)       │  │
│  └─────────────┴─────────────┴──────────────────────────────────┘  │
│  Exemplo: "EngenhariaDeDados" →                                     │
│           [17, 'E', 'n', 'g', 'e', ptr → "nhariaDeDados"]           │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘

VANTAGENS:
1. Tamanho fixo de 16 bytes = cache friendly
2. Comparação rápida: primeiro length, depois prefix
3. Strings curtas não acessam heap
4. Prefixo inline permite early exit em comparações
```

In [ ]:
class GermanString:
    """Simulação da estrutura German String do DuckDB"""
    INLINE_LENGTH = 12  # Strings até 12 bytes ficam inline
    PREFIX_LENGTH = 4   # Primeiros 4 bytes do prefixo
    TOTAL_SIZE = 16     # Tamanho fixo da estrutura
    
    def __init__(self, s: str):
        self.original = s
        self.length = len(s)
        
        if self.length <= self.INLINE_LENGTH:
            # String curta: armazena tudo inline
            self.is_inline = True
            self.inline_data = s.ljust(self.INLINE_LENGTH, '\x00')  # Pad com zeros
            self.prefix = s[:self.PREFIX_LENGTH] if len(s) >= self.PREFIX_LENGTH else s
            self.heap_pointer = None
        else:
            # String longa: armazena prefixo + ponteiro
            self.is_inline = False
            self.inline_data = None
            self.prefix = s[:self.PREFIX_LENGTH]
            self.heap_pointer = s  # Em C++ seria um ponteiro real
    
    def __eq__(self, other: 'GermanString') -> bool:
        """Comparação otimizada - early exit!"""
        # Passo 1: Comparar length (1 instrução - comparação de int32)
        if self.length != other.length:
            return False
        
        # Passo 2: Comparar prefixo (1 instrução - 4 bytes = 1 int32)
        if self.prefix != other.prefix:
            return False
        
        # Passo 3: Strings curtas - comparar inline
        if self.is_inline and other.is_inline:
            return self.inline_data == other.inline_data
        
        # Passo 4: Strings longas - seguir ponteiro e comparar resto
        if self.heap_pointer and other.heap_pointer:
            return self.heap_pointer == other.heap_pointer
        
        return False
    
    def __repr__(self):
        mode = "inline" if self.is_inline else f"heap (prefix='{self.prefix}')"
        return f"GermanString('{self.original}', {mode})"
    
    def memory_layout(self):
        """Mostra layout de memória"""
        if self.is_inline:
            return f"[len={self.length:2}] [{self.inline_data[:12]:12}]"
        else:
            return f"[len={self.length:2}] [prefix='{self.prefix}'] [ptr→'{self.heap_pointer[4:]}']" 

# Demonstração
print("=== Strings Curtas (inline) ===")
short_strings = ["Hi", "Hello", "DuckDB", "12345678901"]
for s in short_strings:
    gs = GermanString(s)
    print(f"{gs}")
    print(f"  Layout: {gs.memory_layout()}")

print("\n=== Strings Longas (heap) ===")
long_strings = ["EngenhariaDeDados", "EngenhariaDeSoftware", "DataEngineeringCourse"]
for s in long_strings:
    gs = GermanString(s)
    print(f"{gs}")
    print(f"  Layout: {gs.memory_layout()}")

## 8.2 Comparação Otimizada: Early Exit

A grande vantagem das German Strings é poder descartar non-matches **sem acessar a heap**:

```
Comparar: "EngenhariaDeDados" vs "EngenhariaDeSoftware"

STRING TRADICIONAL:
  1. Seguir ponteiro A → ler "EngenhariaDeDados"
  2. Seguir ponteiro B → ler "EngenhariaDeSoftware"
  3. Comparar byte a byte
  4. Descobrir diferença no caractere 13
  Cache misses: 2 (um para cada string)

GERMAN STRING:
  1. Comparar length: 17 vs 20 → DIFERENTE!
  2. Retornar False imediatamente
  Cache misses: 0!

Comparar: "Banana" vs "Bacana"

GERMAN STRING:
  1. Comparar length: 6 vs 6 → OK, continuar
  2. Comparar prefix: "Bana" vs "Baca" → DIFERENTE!
  3. Retornar False
  Cache misses: 0 (strings inline)!
```

In [ ]:
# Demonstração de early exit
def compare_with_stats(s1: str, s2: str):
    """Compara strings mostrando estatísticas"""
    gs1 = GermanString(s1)
    gs2 = GermanString(s2)
    
    print(f"\nComparando: '{s1}' vs '{s2}'")
    
    # Passo 1: Length
    print(f"  1. Length: {gs1.length} vs {gs2.length}", end="")
    if gs1.length != gs2.length:
        print(" → DIFERENTE! (early exit)")
        return False
    print(" → OK")
    
    # Passo 2: Prefix
    print(f"  2. Prefix: '{gs1.prefix}' vs '{gs2.prefix}'", end="")
    if gs1.prefix != gs2.prefix:
        print(" → DIFERENTE! (early exit)")
        return False
    print(" → OK")
    
    # Passo 3: Resto
    if gs1.is_inline:
        print(f"  3. Inline data: comparação direta")
    else:
        print(f"  3. Heap: seguindo ponteiro para comparar resto")
    
    result = gs1 == gs2
    print(f"  Resultado: {result}")
    return result

# Exemplos
compare_with_stats("EngenhariaDeDados", "EngenhariaDeSoftware")  # Diferente no length
compare_with_stats("Banana", "Bacana")  # Diferente no prefix
compare_with_stats("Hello", "Hello")  # Iguais (inline)
compare_with_stats("EngenhariaDeDados", "EngenhariaDeDados")  # Iguais (heap)

## 8.3 Benchmark: German String vs String Tradicional

In [ ]:
def random_string(length: int) -> str:
    """Gera string aleatória"""
    return ''.join(random.choices(string.ascii_letters, k=length))

def benchmark_comparison(n: int, string_length: int, match_ratio: float = 0.1):
    """
    Benchmark de comparação de strings.
    match_ratio: fração de pares que são iguais
    """
    # Gerar dados
    strings_a = [random_string(string_length) for _ in range(n)]
    strings_b = [random_string(string_length) for _ in range(n)]
    
    # Algumas iguais
    for i in range(int(n * match_ratio)):
        idx = random.randint(0, n-1)
        strings_b[idx] = strings_a[idx]
    
    # Criar German Strings
    german_a = [GermanString(s) for s in strings_a]
    german_b = [GermanString(s) for s in strings_b]
    
    # Benchmark: Comparação Python padrão
    start = time.perf_counter()
    matches_python = sum(1 for a, b in zip(strings_a, strings_b) if a == b)
    python_time = time.perf_counter() - start
    
    # Benchmark: German String
    start = time.perf_counter()
    matches_german = sum(1 for a, b in zip(german_a, german_b) if a == b)
    german_time = time.perf_counter() - start
    
    return {
        'n': n,
        'string_length': string_length,
        'python_ms': python_time * 1000,
        'german_ms': german_time * 1000,
        'matches': matches_python
    }

# Benchmark com diferentes tamanhos de string
n = 100000
lengths = [5, 10, 15, 20, 50, 100]
results = []

print(f"Benchmark: {n:,} comparações de strings\n")
print(f"{'Length':<10} {'Python (ms)':<15} {'German (ms)':<15} {'Matches'}")
print("-" * 55)

for length in lengths:
    r = benchmark_comparison(n, length, match_ratio=0.05)
    results.append(r)
    print(f"{length:<10} {r['python_ms']:<15.2f} {r['german_ms']:<15.2f} {r['matches']}")

In [ ]:
# Visualização
df_results = pd.DataFrame(results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Tempo por método
x = np.arange(len(lengths))
width = 0.35
ax1.bar(x - width/2, df_results['python_ms'], width, label='Python str', color='#3498db')
ax1.bar(x + width/2, df_results['german_ms'], width, label='German String', color='#27ae60')
ax1.set_xlabel('Tamanho da String')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Tempo de Comparação por Tamanho de String')
ax1.set_xticks(x)
ax1.set_xticklabels(lengths)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Destaque: inline vs heap
colors = ['#27ae60' if l <= 12 else '#e74c3c' for l in lengths]
ax2.bar(range(len(lengths)), df_results['german_ms'], color=colors)
ax2.axvline(x=2.5, color='black', linestyle='--', label='Limite inline (12 bytes)')
ax2.set_xlabel('Tamanho da String')
ax2.set_ylabel('Tempo German String (ms)')
ax2.set_title('German String: Inline (verde) vs Heap (vermelho)')
ax2.set_xticks(range(len(lengths)))
ax2.set_xticklabels(lengths)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8.4 Dictionary Encoding para Strings

Para colunas com muitas repetições, DuckDB usa **Dictionary Encoding**:

```
Dados originais:
  ["São Paulo", "Rio", "São Paulo", "Curitiba", "Rio", "Rio", ...]

Dictionary Encoding:
  Dictionary: {0: "São Paulo", 1: "Rio", 2: "Curitiba"}
  Indices:    [0, 1, 0, 2, 1, 1, ...]

Vantagens:
  1. Armazenamento: strings únicas armazenadas uma vez
  2. Comparação: compara inteiros em vez de strings!
  3. Cache: dicionário pequeno cabe no cache
```

In [ ]:
class DictionaryEncodedColumn:
    """Coluna de strings com Dictionary Encoding"""
    
    def __init__(self):
        self.dictionary = []  # Lista de strings únicas
        self.value_to_idx = {}  # Mapeamento string → índice
        self.indices = []  # Índices para cada valor
    
    def encode(self, values: list) -> 'DictionaryEncodedColumn':
        """Codifica lista de strings"""
        for val in values:
            if val not in self.value_to_idx:
                self.value_to_idx[val] = len(self.dictionary)
                self.dictionary.append(val)
            self.indices.append(self.value_to_idx[val])
        
        self.indices = np.array(self.indices, dtype=np.int32)
        return self
    
    def decode(self, idx: int) -> str:
        """Decodifica índice para string"""
        return self.dictionary[self.indices[idx]]
    
    def filter_equals(self, value: str) -> np.ndarray:
        """Filtra linhas onde coluna == value (vetorizado!)"""
        if value not in self.value_to_idx:
            return np.array([], dtype=int)
        
        target_idx = self.value_to_idx[value]
        return np.where(self.indices == target_idx)[0]  # Comparação de inteiros!
    
    def memory_stats(self, original_values: list):
        """Calcula estatísticas de memória"""
        # Original: soma dos tamanhos das strings
        original_bytes = sum(len(s) for s in original_values)
        
        # Dictionary encoded
        dict_bytes = sum(len(s) for s in self.dictionary)
        indices_bytes = len(self.indices) * 4  # int32
        encoded_bytes = dict_bytes + indices_bytes
        
        return {
            'original_bytes': original_bytes,
            'encoded_bytes': encoded_bytes,
            'compression_ratio': original_bytes / encoded_bytes,
            'unique_values': len(self.dictionary)
        }

# Demonstração
cities = ['São Paulo', 'Rio de Janeiro', 'Curitiba', 'Belo Horizonte', 'Salvador',
          'Fortaleza', 'Brasília', 'Porto Alegre', 'Recife', 'Manaus']

# Simular coluna com muitas repetições
n = 100000
data = [random.choice(cities) for _ in range(n)]

# Codificar
column = DictionaryEncodedColumn().encode(data)

print("=== Dictionary Encoding ===")
print(f"Dicionário ({len(column.dictionary)} entradas):")
for i, s in enumerate(column.dictionary):
    print(f"  {i}: '{s}'")

print(f"\nPrimeiros 20 índices: {column.indices[:20]}")

# Estatísticas
stats = column.memory_stats(data)
print(f"\n=== Estatísticas de Memória ===")
print(f"Original:    {stats['original_bytes']:,} bytes")
print(f"Codificado:  {stats['encoded_bytes']:,} bytes")
print(f"Compressão:  {stats['compression_ratio']:.1f}x")

In [ ]:
# Benchmark: Filtro com e sem dictionary encoding
def filter_string_native(data: list, target: str) -> list:
    """Filtro nativo - compara strings"""
    return [i for i, s in enumerate(data) if s == target]

target = 'São Paulo'

# Native
start = time.perf_counter()
for _ in range(100):
    result_native = filter_string_native(data, target)
native_time = (time.perf_counter() - start) / 100

# Dictionary Encoded
start = time.perf_counter()
for _ in range(100):
    result_encoded = column.filter_equals(target)
encoded_time = (time.perf_counter() - start) / 100

print(f"Filtro WHERE cidade = '{target}' em {n:,} linhas")
print(f"  String nativo:      {native_time*1000:.3f} ms ({len(result_native):,} matches)")
print(f"  Dictionary encoded: {encoded_time*1000:.3f} ms ({len(result_encoded):,} matches)")
print(f"  Speedup:            {native_time/encoded_time:.1f}x")

## 8.5 Operações LIKE com Prefixo

German Strings também otimizam `LIKE 'prefix%'`:

```
WHERE nome LIKE 'Eng%'

Com German Strings:
  1. Extrair prefixo 'Eng' (3 bytes)
  2. Comparar com prefix inline (sem seguir ponteiro!)
  3. Se prefix inline começa com 'Eng': pode ser match
  4. Só então seguir ponteiro para confirmar
```

In [ ]:
def like_prefix_german(strings: list, prefix: str) -> list:
    """
    Simula LIKE 'prefix%' com German Strings.
    Usa early exit no prefixo inline.
    """
    results = []
    prefix_len = len(prefix)
    
    for i, s in enumerate(strings):
        gs = GermanString(s)
        
        # Early exit: se string é menor que prefix, não pode dar match
        if gs.length < prefix_len:
            continue
        
        # Comparar com prefixo inline (primeiros 4 bytes)
        check_len = min(prefix_len, GermanString.PREFIX_LENGTH)
        if gs.prefix[:check_len] != prefix[:check_len]:
            continue  # Early exit!
        
        # Confirmação completa
        if s.startswith(prefix):
            results.append(i)
    
    return results

def like_prefix_native(strings: list, prefix: str) -> list:
    """LIKE 'prefix%' nativo"""
    return [i for i, s in enumerate(strings) if s.startswith(prefix)]

# Teste
test_strings = [
    "EngenhariaDeDados", "EngenhariaDeSoftware", "DataScience",
    "EngenhariaEletrica", "Medicina", "Engenharia Civil",
    "Analytics", "Engineering", "Economia"
] * 10000

prefix = "Eng"

# Native
start = time.perf_counter()
result_native = like_prefix_native(test_strings, prefix)
native_time = time.perf_counter() - start

# German
start = time.perf_counter()
result_german = like_prefix_german(test_strings, prefix)
german_time = time.perf_counter() - start

print(f"LIKE '{prefix}%' em {len(test_strings):,} strings")
print(f"  Nativo:        {native_time*1000:.2f} ms ({len(result_native):,} matches)")
print(f"  German String: {german_time*1000:.2f} ms ({len(result_german):,} matches)")

## 8.6 DuckDB: Operações com Strings em Ação

In [ ]:
con = duckdb.connect(':memory:')

# Criar tabela com strings
con.execute("""
    CREATE TABLE produtos AS
    SELECT 
        i AS id,
        'PROD_' || LPAD(i::VARCHAR, 8, '0') AS codigo,
        CASE (i % 20)
            WHEN 0 THEN 'Eletrônicos e Informática'
            WHEN 1 THEN 'Eletrodomésticos'
            WHEN 2 THEN 'Móveis e Decoração'
            WHEN 3 THEN 'Alimentos e Bebidas'
            WHEN 4 THEN 'Moda e Vestuário'
            WHEN 5 THEN 'Esportes e Lazer'
            WHEN 6 THEN 'Livros e Papelaria'
            WHEN 7 THEN 'Saúde e Beleza'
            WHEN 8 THEN 'Automotivo'
            WHEN 9 THEN 'Ferramentas'
            WHEN 10 THEN 'Jardim e Piscina'
            WHEN 11 THEN 'Pet Shop'
            WHEN 12 THEN 'Bebês'
            WHEN 13 THEN 'Games'
            WHEN 14 THEN 'Instrumentos Musicais'
            WHEN 15 THEN 'Construção'
            WHEN 16 THEN 'Camping'
            WHEN 17 THEN 'Escritório'
            WHEN 18 THEN 'Segurança'
            ELSE 'Outros'
        END AS categoria,
        'Descrição detalhada do produto número ' || i || ' com informações adicionais sobre características e especificações' AS descricao,
        MD5(i::VARCHAR) AS hash_id
    FROM generate_series(1, 5000000) AS t(i)
""")

print("Tabela criada com 5M linhas")
con.execute("SELECT * FROM produtos LIMIT 3").df()

In [ ]:
# Benchmark de operações com strings
queries = {
    'Igualdade (curta)': "SELECT COUNT(*) FROM produtos WHERE categoria = 'Eletrônicos e Informática'",
    'Igualdade (longa)': "SELECT COUNT(*) FROM produtos WHERE descricao = 'Descrição detalhada do produto número 1 com informações adicionais sobre características e especificações'",
    'LIKE prefix%': "SELECT COUNT(*) FROM produtos WHERE codigo LIKE 'PROD_0000%'",
    'LIKE %contains%': "SELECT COUNT(*) FROM produtos WHERE descricao LIKE '%número 123%'",
    'LIKE %suffix': "SELECT COUNT(*) FROM produtos WHERE codigo LIKE '%00001'",
    'LENGTH': "SELECT AVG(LENGTH(descricao)) FROM produtos",
    'UPPER': "SELECT COUNT(*) FROM produtos WHERE UPPER(categoria) = 'ELETRÔNICOS E INFORMÁTICA'",
    'LOWER': "SELECT COUNT(*) FROM produtos WHERE LOWER(categoria) LIKE '%eletr%'",
    'CONCAT': "SELECT COUNT(*) FROM produtos WHERE codigo || '-' || categoria LIKE '%Eletr%'",
    'SUBSTRING': "SELECT COUNT(*) FROM produtos WHERE SUBSTRING(codigo, 1, 7) = 'PROD_00'",
    'REPLACE': "SELECT COUNT(*) FROM produtos WHERE REPLACE(categoria, ' ', '_') LIKE '%Inform%'",
}

print("Benchmark de operações com strings:\n")
string_results = []
for name, query in queries.items():
    times = []
    for _ in range(3):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    
    avg_time = np.mean(times) * 1000
    string_results.append({'operation': name, 'time_ms': avg_time})
    print(f"{name:25} {avg_time:8.2f} ms")

In [ ]:
# Visualização
df_strings = pd.DataFrame(string_results)

# Colorir por tipo de operação
colors = []
for op in df_strings['operation']:
    if 'LIKE' in op:
        colors.append('#e74c3c')  # Vermelho para LIKE
    elif 'Igualdade' in op:
        colors.append('#27ae60')  # Verde para igualdade
    else:
        colors.append('#3498db')  # Azul para outras

plt.figure(figsize=(12, 6))
bars = plt.barh(df_strings['operation'], df_strings['time_ms'], color=colors)
plt.xlabel('Tempo (ms)')
plt.title('Performance de Operações com Strings no DuckDB (5M linhas)')
plt.grid(True, alpha=0.3, axis='x')

for bar, val in zip(bars, df_strings['time_ms']):
    plt.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}ms', va='center')

# Legenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#27ae60', label='Igualdade (rápido)'),
    Patch(facecolor='#e74c3c', label='LIKE (varia)'),
    Patch(facecolor='#3498db', label='Funções de string')
]
plt.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

## 8.7 Comparação: DuckDB vs Pandas

In [ ]:
# Carregar dados para Pandas
df_pandas = con.execute("SELECT codigo, categoria, descricao FROM produtos").df()
print(f"DataFrame: {len(df_pandas):,} linhas")

# Benchmark: Filtro de igualdade
target_category = 'Eletrônicos e Informática'

# DuckDB
times_duck = []
for _ in range(5):
    start = time.perf_counter()
    con.execute(f"SELECT COUNT(*) FROM produtos WHERE categoria = '{target_category}'").fetchall()
    times_duck.append(time.perf_counter() - start)

# Pandas
times_pandas = []
for _ in range(5):
    start = time.perf_counter()
    count = len(df_pandas[df_pandas['categoria'] == target_category])
    times_pandas.append(time.perf_counter() - start)

print(f"\nFiltro: categoria = '{target_category}'")
print(f"DuckDB: {np.mean(times_duck)*1000:.2f} ms")
print(f"Pandas: {np.mean(times_pandas)*1000:.2f} ms")
print(f"Speedup: {np.mean(times_pandas)/np.mean(times_duck):.1f}x")

In [ ]:
# Benchmark: LIKE prefix
prefix = 'PROD_0000'

# DuckDB
times_duck2 = []
for _ in range(5):
    start = time.perf_counter()
    con.execute(f"SELECT COUNT(*) FROM produtos WHERE codigo LIKE '{prefix}%'").fetchall()
    times_duck2.append(time.perf_counter() - start)

# Pandas
times_pandas2 = []
for _ in range(5):
    start = time.perf_counter()
    count = len(df_pandas[df_pandas['codigo'].str.startswith(prefix)])
    times_pandas2.append(time.perf_counter() - start)

print(f"\nFiltro: codigo LIKE '{prefix}%'")
print(f"DuckDB: {np.mean(times_duck2)*1000:.2f} ms")
print(f"Pandas: {np.mean(times_pandas2)*1000:.2f} ms")
print(f"Speedup: {np.mean(times_pandas2)/np.mean(times_duck2):.1f}x")

## 8.8 Resumo

| Característica | String Tradicional | German String |
|----------------|-------------------|---------------|
| **Tamanho** | Variável | Fixo (16 bytes) |
| **Comparação** | Byte a byte | Length → Prefix → Resto |
| **Strings curtas** | Sempre na heap | Inline (≤12 bytes) |
| **Cache** | Imprevisível | Friendly (tamanho fixo) |
| **Early exit** | Não | Sim (length + prefix) |

### Técnicas de Otimização:

1. **German Strings**: Prefixo inline para comparação rápida
2. **Dictionary Encoding**: Compara inteiros em vez de strings
3. **LIKE prefix%**: Usa prefixo inline para early exit
4. **Tamanho fixo**: Permite vetorização e cache eficiente
5. **Strings curtas inline**: Evita acesso à heap

### Quando cada técnica brilha:

| Operação | Técnica Ideal |
|----------|---------------|
| `=` com baixa cardinalidade | Dictionary Encoding |
| `=` com strings únicas | German String |
| `LIKE 'prefix%'` | German String prefix |
| `LIKE '%contains%'` | Scan completo (inevitável) |
| Ordenação | German String prefix comparison |

In [ ]:
con.close()